https://qiita.com/maskot1977/items/c9e96a5ed88017bcb96b

In [1]:
!pip install accelerate
!pip install peft bitsandbytes transformers trl sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [2]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import bitsandbytes as bnb
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
import transformers

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    )


Model from cyber agent.

In [3]:
model_name = "cyberagent/open-calm-large"

Load dataset.

In [4]:
dataset_name = "kunishou/databricks-dolly-15k-ja"
#dataset_name = "mlabonne/guanaco-llama2-1k"

data = load_dataset(dataset_name, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token   # When padding sequences to reach a fixed length, the tokenizer will not separate padding token. This means the padding itself will stop when it reaches the maximum token, to save memory.
tokenizer.padding_side = "right"  # add zeros to the right if the sentence reaches the end.

tokenizer_config.json:   0%|          | 0.00/326 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Add 'input_ids' and 'attention_mask' columns in the dataset, needed for transformers.Trainer() later.  You don't need other columns.

In [7]:
import pandas as pd
df = data.to_pandas()
df.head(1)

,instruction,input,index,output,category,input_ids,attention_mask
0,ヴァージン・オーストラリア航空はいつから運航を開始したのですか？,ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty ...,0,ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の...,closed_qa,"[31105, 260, 286, 5409, 3023, 257, 245, 2887, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [8]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,     # It uses 4 bits format.  It will run faster but may be less accurate.
    bnb_4bit_quant_type="nf4",    # quantization type.
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,    # Use 4 bits for both weights and activations if set true.  Might be less accurate if set true.
    )

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map='auto' ,  #  {"": 0}  #'auto',   # {"": 0} set it to cpu.
    )

model.config.use_cache = False
model.config.pretraining_tp = 1   # This parameter is a type of attention mechanism called "Llama2 Attention" instead of using other attention mechanism.  Settting it other than 1 will be computationary expensive.

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
prompt = "究極生命体カーズとは?"
#prompt = "あなたは誰"

#https://huggingface.co/docs/transformers/internal/generation_utils
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                max_length=100,
                temperature=0.9,
                top_p=0.75,
                top_k=40,
                repetition_penalty=2.0,  # default=5.0
                #num_beams=5,  # default=10
                )

result = pipe(f"{prompt}")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should se

あなたは誰ですか?
あなたの人生で何が欲しいか、そしてそれをどのように手に入れたいか。それはあなたが何を欲しがっているかに基づいています。「私はこれをほしい」と「これは私のものだ!」は違いますよね?「これを手に入れたいけど私には無理だわ...」という場合は、「これが私の望みです!だから私にください!!」と言いたいですよね(笑)でも実はこの2つには違いがあります...なぜならその望みは本当に叶えられるものかどうかを確かめる必要があるからです...。ではなぜそれが叶うのかというと


In [11]:
from peft import get_peft_model

peft_params = LoraConfig(
    lora_alpha=16,   # coefficient that is multiplied to the matrix in low rank approximation.  A higher value might lead to better accuracy but with increased computational cost.
    lora_dropout=0.1,
    r=16,   # Matrix defined in low rank approximation.  It's the matrix dimension rxr that will be updated.  Matrix of weights = V*M(rxr)*W.   A higher rank can lead to better accuracy but requires more memory.
    bias="none",
    task_type="CAUSAL_LM",   # TaskType.SEQ_2_SEQ_LM
    )

model = get_peft_model(model, peft_params)

In [12]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,   # number of weight updates before performing backward propagation.
    optim="paged_adamw_32bit",   # optimizer
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,   # whether to use 16bit or 32bit for training during gradient calculation.
    bf16=False,
    max_grad_norm=0.3,    # same as tensorflow gradient clip.
    max_steps=-1,      # max number of epochs in case if all data is not used for training.
    warmup_ratio=0.03,    #  Ratio of total training steps used for a linear warmup from 0 to learning_rate.
    group_by_length=True,  # Whether or not to group together samples of roughly the same length in the training dataset
    lr_scheduler_type="constant",
    report_to="tensorboard"
    )

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,  #dataset_small,
    #dataset_batch_size=128,
    peft_config=peft_params,
    dataset_text_field="output",       # ********** choose the column for training
    max_seq_length=None,    # Set this to 1024 to avoid warning.
    tokenizer=tokenizer,
    args=training_params,
    packing=False,  # If some text is short, it will combine with other text data for faster training.
    )

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [15]:
#model.config.use_cache = False  # 警告を黙らせます。 推論のために再度有効にしてください。
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,3.348600
50,4.494900
75,3.269600
100,3.952300
125,3.275600
150,3.848200
175,3.225700
200,3.865400
225,3.215700
250,3.869400


TrainOutput(global_step=3754, training_loss=3.518927450693219, metrics={'train_runtime': 672.422, 'train_samples_per_second': 22.33, 'train_steps_per_second': 5.583, 'total_flos': 5403905203027968.0, 'train_loss': 3.518927450693219, 'epoch': 1.0})

This function **trainer.model.save_pretrained()** saves the adapter model and the adapter configuration files to a directory, so that it can be
reloaded using the **PeftModel.from_pretrained(model=, model_id=(hugging_face or local_dir)** class method, and also used by the [PeftModel.push_to_hub]
method.

https://huggingface.co/docs/peft/package_reference/peft_model

https://www.kaggle.com/code/mertsengil/test-mistral

In [16]:
# Save your Fine-tuned model.  You can name it anything.
new_model = model_name.split("/")[1] + "_trained"

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('open-calm-large_trained/tokenizer_config.json',
 'open-calm-large_trained/special_tokens_map.json',
 'open-calm-large_trained/tokenizer.json')

In [45]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                max_length=100,
                temperature=0.99,
                top_p=0.75,
                top_k=40,
                repetition_penalty=2.0,  # default=5.0
                num_beams=3,  # default=10
                )

prompt = "究極生命体カーズとは?"
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

究極生命体カーズとは?
カーズは、マーベル・シネマティック・ユニバース(MCU)に登場するキャラクターです。カーズは、マーベル・シネマティック・ユニバース(MCU)のヒーローチームであるアベンジャーズのメンバーです。カーズは、アイアンマン、キャプテン・アメリカ、ソー、ブラック・ウィドウ、ホークアイ、アントマン、ドクター・ストレンジ、ブラックパンサー、スパイダーマン、ガーディアンズ・オブ・ギャラクシー、マイティ・ソー、キャプテン・アメリカ、ハルクなど、
